In [10]:
import pandas as pd
import ast
import datetime
import subprocess
import sys
import traceback

In [11]:
def gestion_nan(df_log):
    df_log.dropna(inplace=True)
    return df_log

def extract_sensor_data(df_log):
    l_sensor_data = []
    for row in df_log["sensor_data"]:
        dico = ast.literal_eval(row)
        l_sensor_data.append(dico)
    df_sensor_data = pd.DataFrame(l_sensor_data, 
                                columns=["temp","pressure","vibrations"])
    df = pd.concat([df_log,df_sensor_data],axis=1)
    return df

def conversion_str_to_num(df_log):
    try:
        df_log["temp"] = df_log["temp"].str.replace("°C","").astype("float")
        df_log["pressure"] = df_log["pressure"].str.replace("hPa","").astype("float")
        df_log["vibrations_num"] = df_log["vibrations"].str.replace("m/s²","").astype("float")
    except:
        df_log["temp"] = df_log["temp"].str.replace("Â°C","").astype("float")
        df_log["pressure"] = df_log["pressure"].str.replace("hPa","").astype("float")
        df_log["vibrations_num"] = df_log["vibrations"].str.replace("m/sÂ²","").astype("float")
    return df_log

def conversion_date(df_log):
    df_log["jour_vol"] = pd.to_datetime(df_log["jour_vol"], format=format_date)
    return df_log
    
def avions(df):
    df["end_maint"] = df["end_maint"].fillna("Pas en maintenance")
    df["end_maint"] = df["end_maint"].str.replace(" 14:13:00", "")
    df["last_maint"] = df["last_maint"].str.replace(" 00:00:00", "")
    
    df["last_maint"] = pd.to_datetime(df["last_maint"])
    df["debut_service"] = pd.to_datetime(df["debut_service"])
    return df
    



In [12]:
import pandas as pd
import datetime

# Obtenir la date actuelle
format_date = "%Y-%m-%d"

date_dep = datetime.datetime.strptime(datetime.datetime.now().strftime(format_date),format_date)
date_fin = datetime.datetime.strptime("2024-06-02", format_date)
l_fichier = []
df_global_global = pd.DataFrame()
while date_dep >= date_fin :  # Ici, vous devriez peut-être utiliser <= pour itérer jusqu'à la date de fin
    date_dep_str = date_dep.strftime(format_date)
    l_fichier.append(date_dep)  # Pas besoin de strftime, date_actuelle est déjà une chaîne de caractères

    print(date_dep)
    try :
        log_vol = fr"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\fichiers_telecharges\logs_vols_{date_dep_str}.csv"
        degrad  = fr"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\fichiers_telecharges\degradations_{date_dep_str}.csv"
        avion   = fr"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\donnees_depart\aeronefs_2024-06-02.csv"
        compo   = fr"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\donnees_depart\composants_2024-06-02.csv"

        data_logs = pd.read_csv(log_vol)
        data_degrad = pd.read_csv(degrad)

        data_logs_good = gestion_nan(data_logs)
        data_logs_good = conversion_date(data_logs)
        data_logs_good = extract_sensor_data(data_logs)
        data_logs_good = conversion_str_to_num(data_logs_good)


        data_global = data_degrad.merge(right=data_logs_good, left_on="linked_aero",right_on="aero_linked", how="right")
        data_global.to_csv(f"fichier_fusion_journa/fichier_fusionne_{date_dep_str}.csv")
        print(data_global["measure_day"])
        df_global_global = pd.concat([data_global, df_global_global], axis = 1)
        print("ok")
        
    # data_global.to_csv(f"fichier_fusionne_{date_dep_str}.csv")
    except Exception as e:
        print(e)
        traceback.print_exc()

    date_dep = date_dep - datetime.timedelta(days=1)  # Retirer un jour à date_dep

df_global_global.to_csv(f"fichier_global/data_global_no_history.csv")


2024-07-08 00:00:00
[Errno 2] No such file or directory: 'C:\\Users\\bapti\\Desktop\\WCS Python\\Projet 3\\Projet-3-Skyanalytics\\fichiers_telecharges\\logs_vols_2024-07-08.csv'
2024-07-07 00:00:00
[Errno 2] No such file or directory: 'C:\\Users\\bapti\\Desktop\\WCS Python\\Projet 3\\Projet-3-Skyanalytics\\fichiers_telecharges\\logs_vols_2024-07-07.csv'
2024-07-06 00:00:00


Traceback (most recent call last):
  File "C:\Users\bapti\AppData\Local\Temp\ipykernel_10396\1977610326.py", line 22, in <module>
    data_logs = pd.read_csv(log_vol)
                ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\bapti\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\io\parsers\readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\bapti\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\io\parsers\readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\bapti\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\io\parsers\readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\bapti\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\io\

0       2024-07-06
1       2024-07-06
2       2024-07-06
3       2024-07-06
4       2024-07-06
           ...    
4585    2024-07-06
4586    2024-07-06
4587    2024-07-06
4588    2024-07-06
4589    2024-07-06
Name: measure_day, Length: 4590, dtype: object
ok
2024-07-05 00:00:00
0       2024-07-05
1       2024-07-05
2       2024-07-05
3       2024-07-05
4       2024-07-05
           ...    
4765    2024-07-05
4766    2024-07-05
4767    2024-07-05
4768    2024-07-05
4769    2024-07-05
Name: measure_day, Length: 4770, dtype: object
ok
2024-07-04 00:00:00
0       2024-07-04
1       2024-07-04
2       2024-07-04
3       2024-07-04
4       2024-07-04
           ...    
4630    2024-07-04
4631    2024-07-04
4632    2024-07-04
4633    2024-07-04
4634    2024-07-04
Name: measure_day, Length: 4635, dtype: object
ok
2024-07-03 00:00:00
0       2024-07-03
1       2024-07-03
2       2024-07-03
3       2024-07-03
4       2024-07-03
           ...    
3325    2024-07-03
3326    2024-07-03
3327    202

In [13]:
df_avions = pd.read_csv(r"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\donnees_depart\aeronefs_2024-06-02.csv")
avions(df_avions)

,ref_aero,type_model,debut_service,last_maint,en_maintenance,end_maint
0,E170_6353,E170,2000-03-17,2024-03-27,False,Pas en maintenance
1,B767_2803,B767,2020-12-03,2024-06-01,False,Pas en maintenance
2,CRJ700_6943,CRJ700,2015-01-29,2024-06-02,True,2024-06-03
3,A350_1177,A350,2005-08-07,2024-01-04,False,Pas en maintenance
4,B777_4029,B777,2009-01-01,2024-06-01,False,Pas en maintenance
...,...,...,...,...,...,...
227,B737_4369,B737,2010-01-05,2024-06-01,False,Pas en maintenance
228,A340_4021,A340,2004-12-13,2024-05-31,False,Pas en maintenance
229,CRJ900_1740,CRJ900,2003-07-20,2024-06-01,False,Pas en maintenance
230,E170_5551,E170,2021-04-10,2024-06-01,False,Pas en maintenance


In [14]:
df_compo = pd.read_csv(r"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\donnees_depart\composants_2024-06-02.csv")

In [15]:
df_compo.head()

,ref_compo,categorie,aero,desc,lifespan,taux_usure_actuel,cout
0,REAE170-E170_6353-0,Composants Critiques,E170_6353,Réacteur gauche,13045,53.308375,15552
1,REAE170-E170_6353-1,Composants Critiques,E170_6353,Réacteur droit,13626,50.111950,17559
2,SYSE170-E170_6353-2,Composants Critiques,E170_6353,Système de navigation,10541,16.730575,16514
3,ORDE170-E170_6353-3,Composants Critiques,E170_6353,Ordinateur de vol,10441,6.653075,16047
4,AUTE170-E170_6353-4,Composants Critiques,E170_6353,Autopilote,14847,42.033525,19362


In [16]:
#historique composants
df_global_histo = pd.DataFrame()
date_dep = datetime.datetime.strptime(datetime.datetime.now().strftime(format_date),format_date)
date_fin = datetime.datetime.strptime("2024-06-02", format_date)
while date_dep >= date_fin :  # Ici, vous devriez peut-être utiliser <= pour itérer jusqu'à la date de fin
    date_dep_str = date_dep.strftime(format_date)
    try:
        df_update = pd.read_csv(fr"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\fichier_fusion_journa\fichier_fusionne_{date_dep_str}.csv")
        df_update = df_update[["ref_deg","linked_aero","compo_concerned",
                        "usure_nouvelle","measure_day","need_replacement",
                        "ref_vol","time_en_air","etat_voyant","temp","pressure",
                        "vibrations","vibrations_num"]]
        compo = pd.read_csv(fr"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\donnees_depart\composants_2024-06-02.csv")
        compo = compo[["ref_compo","taux_usure_actuel","cout"]]
        df_update = df_update.merge(compo, left_on="compo_concerned", right_on="ref_compo", how="left")
        df_update["usure_jour"] = df_update["taux_usure_actuel"]-df_update["usure_nouvelle"]
        df_global_histo = pd.concat([df_global_histo, df_update])
    except:
        print(f"fichier du {date_dep_str} manquant")
    date_dep = date_dep - datetime.timedelta(days=1)
    
df_global_histo.to_csv("fichier_global/data_global_history.csv")

fichier du 2024-07-08 manquant
fichier du 2024-07-07 manquant


In [17]:
df_global_histo.sort_values(by="ref_deg")

,ref_deg,linked_aero,compo_concerned,usure_nouvelle,measure_day,need_replacement,ref_vol,time_en_air,etat_voyant,temp,pressure,vibrations,vibrations_num,ref_compo,taux_usure_actuel,cout,usure_jour
1125,D000002,A340_4021,REAA340-A340_4021-0,60.781770,2024-06-08,False,V07401178,6.2,0,7.1,997.6,3.8978681384539504 m/s²,3.897868,REAA340-A340_4021-0,59.007650,15552,-1.774120
654,D000002,A340_4633,FREA340-A340_4633-24,23.343865,2024-06-11,False,V00467979,3.9,0,-8.7,1058.7,2.218220439863793 m/s²,2.218220,FREA340-A340_4633-24,21.801765,12664,-1.542100
384,D000002,A340_4633,FREA340-A340_4633-24,28.994560,2024-06-28,False,V05193263,4.8,0,-2.9,919.5,2.4132472862720933 m/s²,2.413247,FREA340-A340_4633-24,21.801765,12664,-7.192795
2970,D000002,A340_4021,REAA340-A340_4021-0,66.391120,2024-06-23,False,V08071624,7.7,0,-8.7,1005.0,4.935560161495292 m/s²,4.935560,REAA340-A340_4021-0,59.007650,15552,-7.383470
4680,D000002,A340_4021,REAA340-A340_4021-0,62.725475,2024-06-16,False,V01664668,4.7,0,-4.7,961.5,3.2442129065840923 m/s²,3.244213,REAA340-A340_4021-0,59.007650,15552,-3.717825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,D009999,E175_4558,GOUE175-E175_4558-26,63.525977,2024-06-21,False,V06209065,3.0,0,0.5,867.8,4.0584859732552525 m/s²,4.058486,GOUE175-E175_4558-26,59.505503,12059,-4.020474
3790,D009999,B737_2262,SYSB737-B737_2262-10,46.637360,2024-07-05,False,V01362078,3.2,0,15.9,1009.5,0.9734308764092581 m/s²,0.973431,SYSB737-B737_2262-10,36.781627,18091,-9.855733
3475,D009999,B737_2262,SYSB737-B737_2262-10,47.412662,2024-07-06,False,V00322437,6.6,0,0.9,967.4,1.893433022908451 m/s²,1.893433,SYSB737-B737_2262-10,36.781627,18091,-10.631035
1646,D009999,E175_4558,GOUE175-E175_4558-26,60.377870,2024-06-07,False,V03299176,3.3,0,3.6,903.2,2.3644307979229238 m/s²,2.364431,GOUE175-E175_4558-26,59.505503,12059,-0.872367


In [18]:
df_global_histo_avion = pd.DataFrame()
date_dep = datetime.datetime.strptime(datetime.datetime.now().strftime(format_date),format_date)
date_fin = datetime.datetime.strptime("2024-06-02", format_date)

while date_dep >= date_fin :  # Ici, vous devriez peut-être utiliser <= pour itérer jusqu'à la date de fin
    date_dep_str = date_dep.strftime(format_date)
    try:
        df_update = pd.read_csv(fr"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\fichier_fusion_journa\fichier_fusionne_{date_dep_str}.csv")
        df_update = df_update[["ref_deg","linked_aero","compo_concerned",
                        "usure_nouvelle","measure_day","need_replacement",
                        "ref_vol","time_en_air","etat_voyant","temp","pressure",
                        "vibrations","vibrations_num"]]
        compo = pd.read_csv(fr"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\donnees_depart\composants_2024-06-02.csv")
        compo = compo[["ref_compo","taux_usure_actuel","cout"]]
        df_update = df_update.merge(compo, left_on="compo_concerned", right_on="ref_compo", how="left")
        df_update["usure_jour"] = df_update["taux_usure_actuel"]-df_update["usure_nouvelle"]
        df_global_histo = pd.concat([df_global_histo, df_update])
    except:
        print(f"fichier du {date_dep_str} manquant")
    date_dep = date_dep - datetime.timedelta(days=1)
    
df_global_histo.to_csv("fichier_global/data_global_history.csv")

fichier du 2024-07-08 manquant
fichier du 2024-07-07 manquant


In [19]:
df_update.head(1)


,ref_deg,linked_aero,compo_concerned,usure_nouvelle,measure_day,need_replacement,ref_vol,time_en_air,etat_voyant,temp,pressure,vibrations,vibrations_num,ref_compo,taux_usure_actuel,cout,usure_jour
0,D001878,B767_4924,REAB767-B767_4924-0,31.518445,2024-06-02,False,V07679725,7.0,0,-5.7,930.6,3.1163905014858932 m/s²,3.116391,REAB767-B767_4924-0,31.518445,15552,0.0


In [20]:
df_compo[df_compo["ref_compo"]=="REAA320-A320_6678-0"]

,ref_compo,categorie,aero,desc,lifespan,taux_usure_actuel,cout
315,REAA320-A320_6678-0,Composants Critiques,A320_6678,Réacteur gauche,13045,7.390555,15552
